# SAPO Baseline: 5 GPT-2 Nodes (I=4, J=0) on Single A100 80GB - NO SWARM

This notebook runs the **BASELINE** experiment with **NO swarm collaboration** (standard RL training).

**Configuration:**
- **I=4** (local rollouts per round)
- **J=0** (NO external rollouts - baseline)
- **G=8** (completions per question)
- **Model**: GPT-2 (124M params)
- **Hardware**: 5 nodes (1 coordinator + 4 workers) on 1× A100 80GB

**Purpose:** Establish baseline performance WITHOUT swarm sharing.

**Default Mode: TESTING (10 rounds, ~6 minutes)**
- Quick validation to ensure everything works
- See Cell 2 to switch to PRODUCTION mode (2000 rounds, ~21 hours)

**Expected Results (Production Mode):**
- Cumulative reward: **250-350** (baseline reference)
- Paper (Qwen2.5-0.5B) baseline: 562
- GPT-2 weaker → lower absolute score, but this is the reference for improvement

**Memory Usage:** ~30 GB peak VRAM (4 workers train, coordinator doesn't)

**Paper Reference:** arXiv:2509.08721 - SAPO (Gensyn AI Team, 2025)

**IMPORTANT:** Run this FIRST before other configs to establish baseline!

## 1. Configuration

**This notebook is pre-configured for BASELINE (I=4, J=0) - NO SWARM.**

**⚠️ IMPORTANT: Testing Mode Enabled by Default**
- Default: 10 rounds (~6 minutes) - validates everything works
- Production: 2000 rounds (~21 hours) - full training

**To switch to production mode:** Change `MAX_ROUNDS` in Cell 2 below.

In [ ]:
# SAPO Baseline Experiment Configuration (I=4, J=0) - NO SWARM
# This notebook runs 5 nodes WITHOUT swarm collaboration (standard RL)

# ============================================
# PRE-CONFIGURED FOR BASELINE (NO SWARM)
# ============================================
EXPERIMENT_NAME = 'sapo_gpt2_baseline_4loc0ext'
NUM_TRAIN_SAMPLES = 4        # I: Local rollouts per round
NUM_TRANSPLANT_TREES = 0     # J: NO external rollouts (baseline)

# ============================================
# TRAINING MODE: TESTING (default) or PRODUCTION
# ============================================
# TESTING MODE (default): Quick validation ~6 minutes
MAX_ROUNDS = 10              # Testing: 10 rounds to verify everything works

# PRODUCTION MODE: Full training ~21 hours
# Uncomment line below for production run:
# MAX_ROUNDS = 2000          # Production: Full 2000 rounds (like paper)

# ============================================
# FIXED SETTINGS (same for all experiments)
# ============================================
NUM_NODES = 5                # Run 5 nodes (1 coordinator + 4 workers)
MODEL_NAME = 'gpt2'          # GPT-2 (124M params, fits memory)
NUM_GENERATIONS = 8          # G: Completions per question (like paper)
SEED = 42                    # For reproducibility

# Rollout Sharing Configuration (not used for baseline, but keep for consistency)
ROLLOUT_PUBLISH_FREQUENCY = 'stage'  # When to share rollouts
ROLLOUT_CLEANUP_ENABLED = True       # Enable cleanup to save space
ROLLOUT_KEEP_LAST_N_ROUNDS = 20      # Keep recent rollouts only
ROLLOUT_ARCHIVE_OLD = False          # Don't archive (saves space)

# Checkpoint Configuration
CHECKPOINT_INTERVAL = 100    # Save checkpoints every 100 rounds (~1 hour in production)
                             # Allows resume if training crashes
MAX_STAGES = 1               # Stages per round (1=default)

# Optional: HuggingFace Token
HUGGINGFACE_TOKEN = None  # Set to your token or keep None

# ============================================
# DISPLAY CONFIGURATION
# ============================================
mode = "TESTING" if MAX_ROUNDS <= 20 else "PRODUCTION"
estimated_time = "~6 minutes" if MAX_ROUNDS <= 20 else "~21 hours"

print("="*60)
print(f"SAPO Baseline Experiment (NO SWARM) - {mode} MODE")
print("="*60)
print(f"✓ Mode: {mode}")
print(f"✓ Nodes: {NUM_NODES} (1 coordinator + 4 workers on single A100 80GB)")
print(f"✓ Model: {MODEL_NAME}")
print(f"✓ Config: I={NUM_TRAIN_SAMPLES}, J={NUM_TRANSPLANT_TREES}, G={NUM_GENERATIONS}")
print(f"✓ Experiment: {EXPERIMENT_NAME}")
print(f"✓ Max Rounds: {MAX_ROUNDS}")
print(f"✓ Checkpoints: Every {CHECKPOINT_INTERVAL} rounds")
print()
print(f"Expected VRAM: ~30 GB peak (4 workers train, coordinator doesn't)")
print(f"Expected Time: {estimated_time}")
print()

if mode == "TESTING":
    print("🧪 TESTING MODE ENABLED")
    print("   Quick validation run - verifies:")
    print("   ✓ All nodes start successfully")
    print("   ✓ Training progresses through rounds")
    print("   ✓ Logs are saved to Google Drive")
    print()
    print("   After validation succeeds, uncomment production line")
    print("   in this cell to run full 2000-round training.")
else:
    print("📊 PRODUCTION MODE - Baseline (NO swarm)")
    print("   Expected cumulative reward: 250-350")
    print("   This establishes reference for comparison")
    print("   Paper baseline (Qwen2.5): 562")
    print()
    print("   ⚠️  Run this FIRST before Config 1/2/3!")

print("="*60)

## 2. Mount Google Drive

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set base path (MUST BE SAME ACROSS ALL NODES)
GDRIVE_BASE_PATH = '/content/drive/MyDrive/rl-swarm'
os.makedirs(GDRIVE_BASE_PATH, exist_ok=True)

print(f"✓ Google Drive mounted at: {GDRIVE_BASE_PATH}")

## 3. System Setup & GPU Verification

In [ ]:
import torch

print("="*60)
print("GPU Verification")
print("="*60)

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9

    print(f"✓ GPU: {gpu_name}")
    print(f"✓ Total VRAM: {total_memory:.1f} GB")
    print()

    # Check if we have enough memory
    # Workers: 4 × 6.5 GB = 26 GB (training workload)
    # Coordinator: ~4 GB (model only, no training overhead)
    # Total: 30 GB
    required_memory = (NUM_NODES - 1) * 6.5 + 4  # 4 workers + coordinator
    print(f"Memory Requirements:")
    print(f"  Workers: {NUM_NODES - 1} × 6.5 GB = {(NUM_NODES - 1) * 6.5:.1f} GB")
    print(f"  Coordinator: ~4 GB (non-training)")
    print(f"  Total estimated: {required_memory:.1f} GB")
    print(f"  Available: {total_memory:.1f} GB")
    print(f"  Margin: {total_memory - required_memory:.1f} GB")
    print()

    if total_memory < required_memory:
        print("⚠️  WARNING: Insufficient VRAM!")
        print(f"   Need at least {required_memory:.0f} GB, but have {total_memory:.1f} GB")
        print(f"   Consider reducing NUM_NODES")
        raise RuntimeError("Insufficient GPU memory")
    elif total_memory < 65:
        print("⚠️  WARNING: Tight fit!")
        print(f"   Have {total_memory:.1f} GB. May work but monitor memory closely.")
        print(f"   Recommended: A100 80GB (or reduce NUM_NODES)")
    else:
        print(f"✅ Sufficient VRAM for {NUM_NODES} nodes ({NUM_NODES - 1} training + 1 coordinator)")
else:
    raise RuntimeError("No GPU detected! Select A100 GPU runtime: Runtime > Change runtime type > A100 GPU")

## 4. Clone Repository & Install Dependencies

In [ ]:
%cd /content

# Remove existing directory if it exists
if os.path.exists('/content/rl-swarm'):
    print("Removing existing repository...")
    !rm -rf /content/rl-swarm

# Clone fresh copy
print("Cloning repository...")
!git clone https://github.com/Elrashid/rl-swarm.git /content/rl-swarm

# Change to repo directory
%cd /content/rl-swarm

# Verify clone worked
if not os.path.exists('requirements.txt'):
    raise FileNotFoundError("Repository clone failed - requirements.txt not found")

print("✓ Repository cloned successfully")
print()

# Install dependencies
print("Installing dependencies (this may take 3-5 minutes)...")
print("Note: Warnings about protobuf versions can be ignored")
print()

# Install main dependencies
!pip install -q -r requirements.txt

# Fix protobuf version explicitly
!pip install -q 'protobuf>=4.25.0,<5.0'

# Verify reasoning-gym was installed
try:
    import reasoning_gym
    print()
    print("✓ Dependencies installed successfully")
    print("✓ reasoning-gym verified")
    print("✓ GenRL vendored locally (no separate install needed)")
except ImportError as e:
    print()
    print("❌ ERROR: reasoning-gym failed to install!")
    print("   Please report this issue with the error above")
    raise

## 5. Initialize Experiment

In [ ]:
from rgym_exp.utils.experiment_manager import init_experiment

# Initialize experiment structure in Google Drive
config_overrides = {
    'training.max_round': MAX_ROUNDS,
    'training.num_generations': NUM_GENERATIONS,
    'training.num_transplant_trees': NUM_TRANSPLANT_TREES,
    'training.num_train_samples': NUM_TRAIN_SAMPLES,
    'training.seed': SEED,
}

init_experiment(
    gdrive_base_path=GDRIVE_BASE_PATH,
    experiment_name=EXPERIMENT_NAME,
    config_overrides=config_overrides
)

print(f"✓ Experiment initialized: {EXPERIMENT_NAME}")
print(f"  Path: {GDRIVE_BASE_PATH}/experiments/{EXPERIMENT_NAME}")
print(f"  Config: I={NUM_TRAIN_SAMPLES}, J={NUM_TRANSPLANT_TREES}, G={NUM_GENERATIONS}")
print()
print("⚠️  BASELINE: J=0 means NO swarm sharing (standard RL training)")

## 6. Launch 5-Node Swarm (KEY CELL)

**This cell:**
1. Spawns 5 separate Python processes
2. Each process runs `swarm_launcher.py` with unique NODE_ID
3. All processes share GPU 0 (CUDA_VISIBLE_DEVICES=0)
4. Coordinator (node_0) manages round progression
5. Workers (node_1-4) follow coordinator

**Note for Baseline:** Even though J=0 (no external rollouts), we still run 5 nodes for fair comparison. Each node trains independently without sharing.

**Logs:** Each node writes to Google Drive at `{GDRIVE_BASE_PATH}/experiments/{EXPERIMENT_NAME}/logs/`

**Monitor:** Use next cell (Cell 7) to track progress in real-time

In [ ]:
import subprocess
import os
import time
from datetime import datetime

mode_label = "TESTING" if MAX_ROUNDS <= 20 else "PRODUCTION"
estimated_duration = "~6 minutes" if MAX_ROUNDS <= 20 else "~21 hours"

print("="*60)
print(f"Launching {NUM_NODES}-Node Baseline ({mode_label} MODE)")
print("="*60)
print(f"Experiment: {EXPERIMENT_NAME}")
print(f"Model: {MODEL_NAME}")
print(f"Config: I={NUM_TRAIN_SAMPLES}, J={NUM_TRANSPLANT_TREES} (NO SWARM), G={NUM_GENERATIONS}")
print(f"Rounds: {MAX_ROUNDS} ({estimated_duration})")
print(f"Hardware: All {NUM_NODES} nodes on single GPU (A100 80GB)")
print("="*60)
print()

# =========================================
# CODE VERSION CHECK
# =========================================
print("Checking code version...")
result = subprocess.run(['git', 'log', '--oneline', '-5'],
                       capture_output=True, text=True, cwd='/content/rl-swarm')
commits = result.stdout.strip().split('\n')
latest_commit = commits[0] if commits else "unknown"

print(f"Latest commit: {latest_commit}")
print(f"Recent commits:")
for commit in commits[:3]:
    print(f"  {commit}")
print()
print("="*60)
print("Starting node processes...")
print("="*60)
print()

processes = []
process_stderr = []  # Store stderr for each process
start_time = time.time()  # For ETA calculation

for node_id in range(NUM_NODES):
    # Environment variables for this node
    env = os.environ.copy()
    env['NODE_ID'] = f'node_{node_id}'
    env['NODE_ROLE'] = 'coordinator' if node_id == 0 else 'worker'
    env['MODEL_NAME'] = MODEL_NAME
    env['NUM_TRAIN_SAMPLES'] = str(NUM_TRAIN_SAMPLES)
    env['NUM_TRANSPLANT_TREES'] = str(NUM_TRANSPLANT_TREES)
    env['NUM_GENERATIONS'] = str(NUM_GENERATIONS)
    env['MAX_ROUNDS'] = str(MAX_ROUNDS)
    env['EXPERIMENT_NAME'] = EXPERIMENT_NAME
    env['GDRIVE_PATH'] = GDRIVE_BASE_PATH
    env['CUDA_VISIBLE_DEVICES'] = '0'  # All nodes share GPU 0
    env['SEED'] = str(SEED + node_id)  # Different seed per node (diversity)
    env['ROLLOUT_PUBLISH_FREQUENCY'] = ROLLOUT_PUBLISH_FREQUENCY
    env['ROLLOUT_CLEANUP_ENABLED'] = str(ROLLOUT_CLEANUP_ENABLED)
    env['ROLLOUT_KEEP_LAST_N_ROUNDS'] = str(ROLLOUT_KEEP_LAST_N_ROUNDS)
    env['ROLLOUT_ARCHIVE_OLD'] = str(ROLLOUT_ARCHIVE_OLD)
    env['CHECKPOINT_INTERVAL'] = str(CHECKPOINT_INTERVAL)
    env['MAX_STAGES'] = str(MAX_STAGES)

    if HUGGINGFACE_TOKEN:
        env['HUGGINGFACE_ACCESS_TOKEN'] = HUGGINGFACE_TOKEN

    # Launch process with stderr/stdout capture
    import sys
    process = subprocess.Popen(
        [sys.executable, '-m', 'rgym_exp.runner.swarm_launcher'],
        env=env,
        cwd='/content/rl-swarm',
        stderr=subprocess.PIPE,  # Capture errors
        stdout=subprocess.PIPE,  # Capture output
        text=True
    )
    processes.append(process)
    process_stderr.append(process.stderr)

    role = "COORDINATOR" if node_id == 0 else "WORKER     "
    print(f"✓ Started node_{node_id} ({role}) - PID: {process.pid:5d}")

    # Stagger startup: coordinator gets 10s, workers get 5s
    delay = 10 if node_id == 0 else 5
    time.sleep(delay)

print()
print("="*60)
print("PROCESS HEALTH CHECK")
print("="*60)
print("Waiting 30 seconds for processes to initialize...")
time.sleep(30)

crashed_nodes = []
for i, p in enumerate(processes):
    returncode = p.poll()
    if returncode is not None:
        crashed_nodes.append(i)
        print(f"❌ node_{i}: CRASHED (exit code {returncode})")
        # Try to read stderr
        try:
            stderr_output = process_stderr[i].read()
            if stderr_output:
                print(f"   Error output:")
                for line in stderr_output.split('\n')[-10:]:  # Last 10 lines
                    if line.strip():
                        print(f"     {line}")
        except:
            print(f"   (Could not read error output)")
    else:
        print(f"✓ node_{i}: RUNNING")

print()

if crashed_nodes:
    print(f"⚠️  WARNING: {len(crashed_nodes)}/{NUM_NODES} nodes crashed!")
    print(f"   Crashed: {', '.join(f'node_{i}' for i in crashed_nodes)}")
    print(f"   Check logs in: {GDRIVE_BASE_PATH}/experiments/{EXPERIMENT_NAME}/logs/")
    print()
    user_input = input("Continue with remaining nodes? (yes/no): ")
    if user_input.lower() != 'yes':
        print("Terminating all processes...")
        for p in processes:
            if p.poll() is None:
                p.terminate()
        raise RuntimeError(f"{len(crashed_nodes)} nodes crashed - see errors above")
else:
    print(f"✅ All {NUM_NODES} nodes launched successfully!")

print()
print(f"✓ Training will run for {estimated_duration} ({MAX_ROUNDS} rounds)")
print(f"✓ Logs location: {GDRIVE_BASE_PATH}/experiments/{EXPERIMENT_NAME}/logs/")
print(f"✓ Checkpoints: Every {CHECKPOINT_INTERVAL} rounds")
print()
if MAX_ROUNDS <= 20:
    print("🧪 TESTING MODE: After completion, check Cell 8 for results")
    print("   Then uncomment production line in Cell 2 for full training")
else:
    print("⚠️  Keep this notebook open (browser tab active)")
    print("⚠️  Colab may disconnect after 12-24 hours")
    print("⚠️  Training will continue - use Cell 7.5 to check progress after reconnect")
print()
print("Monitor progress in Cell 7 below...")

## 7. Monitor Training Progress

In [ ]:
import time
from IPython.display import clear_output
import pandas as pd

print("Starting training monitor...")
print("Press 'Stop' button or Ctrl+C to interrupt\n")

monitor_start_time = time.time()

try:
    while True:
        clear_output(wait=True)

        # Check process status
        running = sum(1 for p in processes if p.poll() is None)
        completed = NUM_NODES - running

        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        elapsed_hours = (time.time() - start_time) / 3600

        print("="*70)
        print(f" SAPO Training Monitor - {EXPERIMENT_NAME}")
        print(f" Time: {current_time} | Elapsed: {elapsed_hours:.1f}h")
        print("="*70)
        print()

        # Node status
        print(f"Nodes: {running}/{NUM_NODES} running, {completed} completed")
        print()

        # GPU memory
        if torch.cuda.is_available():
            reserved = torch.cuda.memory_reserved(0) / 1e9
            total = torch.cuda.get_device_properties(0).total_memory / 1e9
            utilization = (reserved / total) * 100

            print(f"GPU: {reserved:.1f} / {total:.1f} GB ({utilization:.1f}%)")

            if utilization > 90:
                print(f"  ⚠️  WARNING: High memory usage!")
            elif utilization > 75:
                print(f"  ⚠️  Memory usage elevated")
            print()

        # Training progress
        try:
            from rgym_exp.utils.experiment_manager import get_experiment_status
            status = get_experiment_status(GDRIVE_BASE_PATH, EXPERIMENT_NAME)

            if status:
                current_round = status.get('current_round', 0)
                progress_pct = (current_round / MAX_ROUNDS) * 100

                print(f"Progress:")
                print(f"  Round: {current_round:4d} / {MAX_ROUNDS} ({progress_pct:5.1f}%)")

                # Progress bar
                bar_length = 50
                filled = int(bar_length * progress_pct / 100)
                bar = '█' * filled + '░' * (bar_length - filled)
                print(f"  [{bar}]")

                # ETA
                if current_round > 10:
                    hours_per_round = elapsed_hours / current_round
                    remaining_rounds = MAX_ROUNDS - current_round
                    eta_hours = remaining_rounds * hours_per_round
                    print(f"  ETA: {eta_hours:.1f} hours (~{eta_hours/24:.1f} days)")

                print()

                # ==========================================
                # TOTAL CUMULATIVE REWARD (PRIMARY METRIC)
                # ==========================================
                if 'total_cumulative_reward' in status:
                    cumulative_rewards = status.get('cumulative_rewards', {})
                    total_cumulative = status['total_cumulative_reward']

                    print("┌" + "─"*68 + "┐")
                    print("│" + " TOTAL CUMULATIVE REWARD (SAPO Paper Metric)".center(68) + "│")
                    print("├" + "─"*68 + "┤")
                    print(f"│  TOTAL: {total_cumulative:8.2f}".ljust(69) + "│")

                    if cumulative_rewards:
                        print("│".ljust(69) + "│")
                        print("│  Per-Node Breakdown:".ljust(69) + "│")
                        for node_id in sorted(cumulative_rewards.keys()):
                            node_reward = cumulative_rewards[node_id]
                            pct = (node_reward / total_cumulative * 100) if total_cumulative > 0 else 0
                            line = f"│    {node_id:10s}: {node_reward:7.2f} ({pct:5.1f}%)"
                            print(line.ljust(69) + "│")

                    # Average per round
                    avg_per_round = total_cumulative / current_round if current_round > 0 else 0
                    print("│".ljust(69) + "│")
                    print(f"│  Avg per round: {avg_per_round:6.4f}".ljust(69) + "│")

                    # Comparison to paper benchmarks
                    print("│".ljust(69) + "│")
                    print("│  Paper Benchmarks (Qwen2.5, 2000 rounds):".ljust(69) + "│")
                    print("│    Baseline (8/0):    562".ljust(69) + "│")
                    print("│    Config 2 (4/4):  1,093 (+94%) ⭐".ljust(69) + "│")
                    print("└" + "─"*68 + "┘")
                    print()

                # Recent performance
                try:
                    from rgym_exp.utils.experiment_manager import get_experiment_metrics
                    df = get_experiment_metrics(GDRIVE_BASE_PATH, EXPERIMENT_NAME)

                    if not df.empty:
                        recent_reward = df.tail(10)['my_reward'].mean()
                        print(f"Recent Performance:")
                        print(f"  Avg reward: {recent_reward:6.4f} (last 10 rounds)")
                        print()
                except Exception:
                    pass

        except Exception as e:
            print(f"Progress: Unable to load status ({e})")
            print()

        # Instructions
        print("-"*70)
        print("Press 'Stop' button or Ctrl+C to halt training")
        print(f"Next update in 60 seconds...")

        # Exit if all completed
        if running == 0:
            print()
            print("="*70)
            print("✅ All nodes completed successfully!")
            print("="*70)
            break

        time.sleep(60)

except KeyboardInterrupt:
    print("\n" + "="*70)
    print("⚠️  Training interrupted by user")
    print("="*70)
    print("\nTerminating all node processes...")

    for i, p in enumerate(processes):
        if p.poll() is None:
            print(f"  Stopping node_{i}... (PID: {p.pid})")
            p.terminate()

    time.sleep(5)

    for i, p in enumerate(processes):
        if p.poll() is None:
            print(f"  Force killing node_{i}... (PID: {p.pid})")
            p.kill()

    print("\n✓ All processes terminated")
    print("\n💾 Note: Training state is checkpointed.")
    print("   Re-run this notebook to resume from last checkpoint.")

## 7.5. Check Real-Time Progress from GDrive (Optional)

**Reconnected after disconnect?** Run this cell to check training progress:
- Shows current round for each node
- Displays elapsed time and GPU memory
- Works even if your notebook disconnected

Progress is saved to GDrive every round, logs flush every 30 seconds.

In [ ]:
# === Real-Time Progress Viewer ===
# Run this cell anytime to check progress from GDrive
# Useful if you reconnect after notebook disconnect

import sys
sys.path.append('/content/rl-swarm')

from rgym_exp.utils.experiment_manager import get_experiment_status
from rgym_exp.utils.progress_tracker import get_experiment_progress

# Get comprehensive status including cumulative rewards
status = get_experiment_status(GDRIVE_BASE_PATH, EXPERIMENT_NAME)
progress = get_experiment_progress(GDRIVE_BASE_PATH, EXPERIMENT_NAME)

print("="*70)
print("REAL-TIME PROGRESS FROM GDRIVE (BASELINE)")
print("="*70)
print(f"Experiment: {EXPERIMENT_NAME}")
print(f"Round: {status.get('current_round', 0)} / {MAX_ROUNDS}")
print(f"Active peers: {status.get('active_peers', 0)}")
print()

# Cumulative Rewards (SAPO paper metric - BASELINE)
if 'total_cumulative_reward' in status:
    cumulative_rewards = status.get('cumulative_rewards', {})
    total_cumulative = status['total_cumulative_reward']
    
    print("Cumulative Rewards (BASELINE - SAPO paper metric):")
    print(f"  Total (all nodes): {total_cumulative:7.2f} 💾 SAVE THIS!")
    print()
    print("  💾 Use this number to calculate improvement in Config 1/2/3")
    print(f"     Formula: ((Config - {total_cumulative:.2f}) / {total_cumulative:.2f}) × 100%")
    print()
    if cumulative_rewards:
        print(f"  Per node:")
        for node_id in sorted(cumulative_rewards.keys()):
            print(f"    {node_id:10s}: {cumulative_rewards[node_id]:7.2f}")
    print()
else:
    print("Cumulative rewards: Not available yet (training just started)")
    print()

# Per-node progress details
print("Per-Node Progress:")
for node_id, node_data in progress.get('nodes', {}).items():
    if 'error' in node_data:
        print(f"  {node_id}: {node_data['error']}")
    else:
        latest_round = node_data.get('latest_round', 'N/A')
        elapsed_sec = node_data.get('elapsed_seconds', 0)
        elapsed_hours = elapsed_sec / 3600
        print(f"  {node_id}: Round {latest_round} | {elapsed_hours:.1f}h elapsed")

print()
print("="*70)
print("Note: Progress updates every round. Logs flush every 30s to GDrive.")
print("You can access logs directly in Google Drive even while training!")

## 8. View Baseline Results

**This is your REFERENCE performance - save this number to compare with Config 1/2/3!**

In [ ]:
from rgym_exp.utils.experiment_manager import get_experiment_metrics
import pandas as pd
import matplotlib.pyplot as plt

print("="*70)
print(f"Results: {EXPERIMENT_NAME}")
print("="*70)
print(f"Config: I={NUM_TRAIN_SAMPLES}, J={NUM_TRANSPLANT_TREES}, G={NUM_GENERATIONS}")
print(f"Model: {MODEL_NAME}")
print(f"Nodes: {NUM_NODES}")
print(f"Rounds: {MAX_ROUNDS}")
print()

# Load metrics
df = get_experiment_metrics(GDRIVE_BASE_PATH, EXPERIMENT_NAME)

if not df.empty:
    # Calculate cumulative reward per node
    node_rewards = df.groupby('node_id')['my_reward'].sum().sort_values(ascending=False)
    total_reward = node_rewards.sum()
    
    print("Cumulative Rewards by Node:")
    for node_id, reward in node_rewards.items():
        print(f"  {node_id:10s}: {reward:7.2f}")
    print(f"  {'TOTAL':10s}: {total_reward:7.2f}")
    print()
    
    # Compare to paper's results
    print("Comparison to Paper (Qwen2.5-0.5B):")
    print("-"*70)
    
    # Identify configuration
    if NUM_TRANSPLANT_TREES == 0:
        paper_reward = 562
        config_name = "Baseline (4/0)"  # 5-node setup: 4 local, 0 external
        paper_improvement = "—"
    elif NUM_TRAIN_SAMPLES == 3 and NUM_TRANSPLANT_TREES == 1:
        paper_reward = 854
        config_name = "Config 1 (3/1)"
        paper_improvement = "+52%"
    elif NUM_TRAIN_SAMPLES == 4 and NUM_TRANSPLANT_TREES == 4:
        paper_reward = 1093
        config_name = "Config 2 (4/4) **BEST**"
        paper_improvement = "+94%"
    elif NUM_TRAIN_SAMPLES == 1 and NUM_TRANSPLANT_TREES == 3:
        paper_reward = 946
        config_name = "Config 3 (1/3)"
        paper_improvement = "+68%"
    else:
        paper_reward = None
        config_name = f"Custom ({NUM_TRAIN_SAMPLES}/{NUM_TRANSPLANT_TREES})"
        paper_improvement = "N/A"
    
    print(f"  Configuration: {config_name}")
    if paper_reward:
        print(f"  Paper (Qwen2.5):  {paper_reward:7.2f} ({paper_improvement})")
        if MAX_ROUNDS >= 2000:
            print(f"  Ours (GPT-2):     {total_reward:7.2f} (~{total_reward/paper_reward*100:.1f}% of paper)")
        else:
            print(f"  Ours (GPT-2):     {total_reward:7.2f} (testing run, not comparable)")
    else:
        print(f"  Ours (GPT-2):     {total_reward:7.2f}")
    print()
    
    # For baseline, show expected improvements for other configs
    if NUM_TRANSPLANT_TREES == 0 and MAX_ROUNDS >= 2000:
        print("="*70)
        print("⭐ BASELINE REFERENCE ⭐")
        print("="*70)
        print(f"  GPT-2 Baseline: {total_reward:7.2f}")
        print()
        print("💾 SAVE THIS NUMBER!")
        print(f"   Use it to calculate improvement in Config 1/2/3")
        print(f"   Formula: ((Config - Baseline) / Baseline) × 100%")
        print()
        print("Expected Improvements (based on paper):")
        print(f"  Config 1 (3/1): +52% → ~{total_reward * 1.52:.0f}")
        print(f"  Config 2 (4/4): +94% → ~{total_reward * 1.94:.0f} (BEST)")
        print(f"  Config 3 (1/3): +68% → ~{total_reward * 1.68:.0f}")
        print()
        print("GPT-2 predictions (weaker model benefits MORE):")
        print(f"  Config 2: +110-150% → ~{total_reward * 2.3:.0f}")
        print()
    
    # Only plot if we have enough data
    if MAX_ROUNDS >= 20:
        print("Generating plots...")
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # Plot 1: Cumulative reward per node
        for node_id in df['node_id'].unique():
            node_df = df[df['node_id'] == node_id].sort_values('round')
            ax1.plot(node_df['round'], node_df['my_reward'].cumsum(), 
                     label=node_id, alpha=0.7)
        
        ax1.set_xlabel('Round', fontsize=12)
        ax1.set_ylabel('Cumulative Reward', fontsize=12)
        ax1.set_title(f'{config_name} - Cumulative Rewards Over Time', fontsize=14, fontweight='bold')
        ax1.legend(loc='upper left', fontsize=8)
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Average reward per round (smoothed)
        round_avg = df.groupby('round')['my_reward'].mean()
        # Apply moving average (adjust window for short runs)
        window_size = min(100, max(5, MAX_ROUNDS // 20))
        smoothed = round_avg.rolling(window=window_size, center=True).mean()
        
        ax2.plot(round_avg.index, round_avg.values, alpha=0.3, color='gray', label='Raw')
        ax2.plot(smoothed.index, smoothed.values, linewidth=2, color='blue', label=f'Smoothed ({window_size}-round MA)')
        
        ax2.set_xlabel('Round', fontsize=12)
        ax2.set_ylabel('Average Reward', fontsize=12)
        ax2.set_title(f'{config_name} - Average Reward per Round', fontsize=14, fontweight='bold')
        ax2.legend(fontsize=10)
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        # Save plot
        plot_path = f'/content/drive/MyDrive/rl-swarm/results_{EXPERIMENT_NAME}.png'
        plt.savefig(plot_path, dpi=150, bbox_inches='tight')
        print(f"✓ Plot saved to: {plot_path}")
        
        plt.show()
    else:
        print("⚠️  Skipping plots - too few rounds for meaningful visualization")
        print(f"   Run with MAX_ROUNDS >= 20 for plots")
    
else:
    print("❌ No metrics available yet")
    print("   Training may not have started or metrics file is missing")